In [1]:
import pandas as pd

# Load the CSV file (replace 'urls.csv' with your actual file name)
data = pd.read_csv("urls3.csv")


In [2]:
# Display the first few rows to verify
print(data.head())

   ID                                              TITLE  \
0   1  Fed official says weak data caused by weather,...   
1   2  Fed's Charles Plosser sees high bar for change...   
2   3  US open: Stocks fall after Fed official hints ...   
3   4  Fed risks falling 'behind the curve', Charles ...   
4   5  Fed's Plosser: Nasty Weather Has Curbed Job Gr...   

                                                 url          PUBLISHER  \
0  http://www.latimes.com/business/money/la-fi-mo...  Los Angeles Times   
1  http://www.livemint.com/Politics/H2EvwJSK2VE6O...           Livemint   
2  http://www.ifamagazine.com/news/us-open-stocks...       IFA Magazine   
3  http://www.ifamagazine.com/news/fed-risks-fall...       IFA Magazine   
4  http://www.moneynews.com/Economy/federal-reser...          Moneynews   

  CATEGORY                          STORY             HOSTNAME     TIMESTAMP  
0        b  ddUyU0VZz0BRneMioxUPQVP6sIxvM      www.latimes.com  1.390000e+12  
1        b  ddUyU0VZz0BRneMiox

In [3]:
data.describe()

,ID,TIMESTAMP
count,19.000000,1.900000e+01
mean,10.000000,1.390000e+12
std,5.627314,0.000000e+00
min,1.000000,1.390000e+12
25%,5.500000,1.390000e+12
50%,10.000000,1.390000e+12
75%,14.500000,1.390000e+12
max,19.000000,1.390000e+12


In [4]:
data.head()

,ID,TITLE,url,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1.390000e+12
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1.390000e+12
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1.390000e+12
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1.390000e+12
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1.390000e+12


In [5]:
# Check if the 'url' column exists
if 'url' not in data.columns:
    raise ValueError("The CSV file must contain a column named 'url'.")

In [6]:
pip install newspaper3k requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install lxml[html_clean]

Note: you may need to restart the kernel to use updated packages.


In [23]:
from newspaper import Article
import requests
from bs4 import BeautifulSoup

In [24]:
pip install aiohttp


Note: you may need to restart the kernel to use updated packages.


In [10]:
import asyncio
import aiohttp
from newspaper import Article
from bs4 import BeautifulSoup




newspaper3k failed for http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss: Article `download()` failed with 400 Client Error: Bad Request for url: https://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story%5C?track=rss on URL http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss
newspaper3k failed for http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436: Article `download()` failed with 404 Client Error: Not Found for url: https://ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436 on URL http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436
newspaper3k failed for http://www.ifamagazine.com/news/fed-risks-falling-behind-the-curve-charles-pl

In [25]:
# Asynchronous function to download and parse articles
async def get_article_content_async(url, session):
    try:
        # Try using newspaper3k for structured articles
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"newspaper3k failed for {url}: {e}")

        # Fall back to BeautifulSoup for other pages
        try:
            async with session.get(url) as response:
                soup = BeautifulSoup(await response.text(), 'html.parser')
                paragraphs = soup.find_all('p')
                return " ".join([p.get_text() for p in paragraphs])
        except Exception as e:
            print(f"BeautifulSoup failed for {url}: {e}")
            return ""



In [26]:
# Function to scrape multiple URLs concurrently
async def scrape_articles(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [get_article_content_async(url, session) for url in urls]
        return await asyncio.gather(*tasks)

# Convert the URLs column to a list and run the async scraper
urls = data['url'].tolist()

# Instead of asyncio.run(), use await directly
contents = await scrape_articles(urls)

# Assign the scraped content back to the DataFrame
data['content'] = contents

# Save the intermediate results
data.to_csv("articles_with_content.csv", index=False)
print("Scraping completed asynchronously! Content saved to 'articles_with_content.csv'.")

newspaper3k failed for http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss: Article `download()` failed with 400 Client Error: Bad Request for url: https://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story%5C?track=rss on URL http://www.latimes.com/business/money/la-fi-mo-federal-reserve-plosser-stimulus-economy-20140310,0,1312750.story\?track=rss
newspaper3k failed for http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436: Article `download()` failed with 404 Client Error: Not Found for url: https://ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436 on URL http://www.ifamagazine.com/news/us-open-stocks-fall-after-fed-official-hints-at-accelerated-tapering-294436
newspaper3k failed for http://www.ifamagazine.com/news/fed-risks-falling-behind-the-curve-charles-pl

In [30]:
# List of trusted domains
trusted_domains = ["bbc.com", "cnn.com", "nytimes.com", "reuters.com"]

# Function to check if the URL is trusted
def is_trusted(url):
    return any(domain in url for domain in trusted_domains)

# Add a column for trustworthiness
data['is_trusted'] = data['url'].apply(is_trusted)

print(data[['url', 'is_trusted']])


                                                  url  is_trusted
0   http://www.latimes.com/business/money/la-fi-mo...       False
1   http://www.livemint.com/Politics/H2EvwJSK2VE6O...       False
2   http://www.ifamagazine.com/news/us-open-stocks...       False
3   http://www.ifamagazine.com/news/fed-risks-fall...       False
4   http://www.moneynews.com/Economy/federal-reser...       False
5   http://www.nasdaq.com/article/plosser-fed-may-...       False
6   http://www.marketwatch.com/story/feds-plosser-...       False
7   http://www.fxstreet.com/news/forex-news/articl...       False
8   http://economictimes.indiatimes.com/news/inter...       False
9   http://www.iii.co.uk/news-opinion/reuters/news...       False
10  http://in.reuters.com/article/2014/03/10/us-ec...        True
11  http://blogs.reuters.com/hugo-dixon/2014/03/10...        True
12  http://in.reuters.com/article/2014/03/10/eu-ba...        True
13  http://in.reuters.com/article/2014/03/10/ecb-p...        True
14        

In [31]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
from transformers import pipeline

# Load pre-trained model with truncation enabled
classifier = pipeline("text-classification", model="mrm8488/bert-tiny-finetuned-fake-news-detection", truncation=True)

# Function to classify article content
def classify_news(content):
    if not content or len(content.strip()) == 0:
        return "Unknown", 0.0  # Handle missing content
    result = classifier(content, truncation=True, max_length=512)[0]
    return result['label'], result['score']

# Apply classification
data['prediction'], data['confidence'] = zip(*data['content'].apply(classify_news))

print(data[['url', 'is_trusted', 'prediction', 'confidence']])

pytorch_model.bin:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

                                                  url  is_trusted prediction  \
0   http://www.latimes.com/business/money/la-fi-mo...       False    Unknown   
1   http://www.livemint.com/Politics/H2EvwJSK2VE6O...       False    LABEL_0   
2   http://www.ifamagazine.com/news/us-open-stocks...       False    LABEL_1   
3   http://www.ifamagazine.com/news/fed-risks-fall...       False    LABEL_1   
4   http://www.moneynews.com/Economy/federal-reser...       False    LABEL_1   
5   http://www.nasdaq.com/article/plosser-fed-may-...       False    LABEL_1   
6   http://www.marketwatch.com/story/feds-plosser-...       False    LABEL_1   
7   http://www.fxstreet.com/news/forex-news/articl...       False    Unknown   
8   http://economictimes.indiatimes.com/news/inter...       False    LABEL_1   
9   http://www.iii.co.uk/news-opinion/reuters/news...       False    LABEL_1   
10  http://in.reuters.com/article/2014/03/10/us-ec...        True    LABEL_1   
11  http://blogs.reuters.com/hugo-dixon/

In [22]:
# Save the results to a new CSV
data.to_csv("news_with_predictions.csv", index=False)
print("Results saved to 'news_with_predictions.csv'.")


Results saved to 'news_with_predictions.csv'.
